In [2]:
import cv2
import pydirectinput
import mediapipe as mp

In [22]:
import pyautogui

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose

In [25]:
cap = cv2.VideoCapture(0)
cap.set(3,720) #width
cap.set(4,540) #height
pose = ""
status = 0
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        _, frame = cap.read()
        frame = cv2.flip(frame,1) # left hand <-> right hand 
        img = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        results = holistic.process(img)
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        
        height,width,_ = img.shape
        
#         cv2.imshow("Image",img)
        try:
            right_hand = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x*width,
                          results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y*height)
            line_x1 = width//3
            line_x2 = 2*(width//3)
            line_y1 = height//3
            line_y2 = 2*(height//3)
            
            if right_hand[0]>line_x2 and right_hand[1]<line_y1 :
                pose = 'start'
                status = 1
                pyautogui.keyDown('space')
                pyautogui.keyUp('space')
            elif right_hand[0]>line_x2 and right_hand[1]>line_y1 and right_hand[1]<line_y2 and status ==1:
                pose = 'Right'
                pyautogui.keyDown('right')
                pyautogui.keyUp('right')
            elif right_hand[0]<line_x1 and right_hand[1]>line_y1 and right_hand[1]<line_y2 and status ==1:
                pose = 'Left'  
                pyautogui.keyDown('left')
                pyautogui.keyUp('left')
            elif right_hand[0]>line_x1 and right_hand[0]<line_x2 and right_hand[1]<line_y1 and status ==1:
                pose = 'jump'  
                pyautogui.keyDown('up')
                pyautogui.keyUp('up')
                
            elif right_hand[0]>line_x1 and right_hand[0]<line_x2 and right_hand[1]>line_y2 and status ==1:
                pose = 'down' 
                pyautogui.keyDown('down')
                pyautogui.keyUp('down')
                
            elif status ==0:
                pose ='Play or Resume'
            else:
                pose = 'Run'
                
        except:
            pass
        
        cv2.putText(img,pose,(width//3,height//3),cv2.FONT_HERSHEY_PLAIN,2,(252,252,52),3)
        cv2.line(img,(width//3,0),(width//3,height),(52,52,52),2)
        cv2.line(img,(2*(width//3),0),(2*(width//3),height),(52,52,52),2)
        cv2.line(img,(0,height//3),(width,height//3),(52,52,52),2)
        cv2.line(img,(0,2*(height//3)),(width,2*(height//3)),(52,52,52),2)
        
        mp_drawing.draw_landmarks(img,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(180,105,255),thickness =5,circle_radius=8),
                                 mp_drawing.DrawingSpec(color = (255,255,255),thickness = 10, circle_radius=10))
        
        cv2.imshow("Temple Run",img)
        
        if cv2.waitKey(1) & 0xff==ord('q'):
            break
        
        
cap.release()
cv2.destroyAllWindows()
        
               

In [16]:
cap.release()
cv2.destroyAllWindows()      